## Example two-layer classifier models

Below example code is given for creating instances of the CIFAR-10 and CIFAR-100 data provider objects and using them to train simple two-layer feedforward network models with rectified linear activations in TensorFlow. You may wish to use this code as a starting point for your own experiments.

In [1]:
import os
import tensorflow as tf
import numpy as np
from mlp.data_providers import CIFAR10DataProvider, CIFAR100DataProvider
import matplotlib.pyplot as plt
%matplotlib inline

/Users/EJ/miniconda2/envs/mlp/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


### CIFAR-10

In [2]:
train_data = CIFAR10DataProvider('train', batch_size=50)
valid_data = CIFAR10DataProvider('valid', batch_size=50)

In [3]:
def fully_connected_layer(inputs, input_dim, output_dim, nonlinearity=tf.nn.relu):
    weights = tf.Variable(
        tf.truncated_normal(
            [input_dim, output_dim], stddev=2. / (input_dim + output_dim)**0.5), 
        'weights')
    biases = tf.Variable(tf.zeros([output_dim]), 'biases')
    outputs = nonlinearity(tf.matmul(inputs, weights) + biases)
    return outputs

In [26]:
def build_model(num_layers):
    inputs = tf.placeholder(tf.float32, [None, train_data.inputs.shape[1]], 'inputs')
    targets = tf.placeholder(tf.float32, [None, train_data.num_classes], 'targets')
    
    num_hidden = 200
    
    lay = dict()
    
    with tf.name_scope('fc-layer-1'):
        lay['fc-layer-1'] = fully_connected_layer(inputs, train_data.inputs.shape[1], num_hidden)
    
    for layer in range(num_layers):
        with tf.name_scope('fc-layer-{}'.format(layer+2)):
            lay['fc-layer-{}'.format(layer+2)] = fully_connected_layer(lay['fc-layer-{}'.format(layer+1)], num_hidden, num_hidden)
    
#     print(lay)
        
    with tf.name_scope('output-layer'):
        outputs = fully_connected_layer(lay['fc-layer-{}'.format(num_layers+1)], num_hidden, train_data.num_classes, tf.identity)

    with tf.name_scope('error'):
        error = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(outputs, targets))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(
                tf.equal(tf.argmax(outputs, 1), tf.argmax(targets, 1)), 
                tf.float32))

    with tf.name_scope('train'):
        train_step = tf.train.AdamOptimizer().minimize(error)

    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        for e in range(5):
            running_error = 0.
            running_accuracy = 0.

            for input_batch, target_batch in train_data:
                _, batch_error, batch_acc = sess.run(
                    [train_step, error, accuracy], 
                    feed_dict={inputs: input_batch, targets: target_batch})

                running_error += batch_error
                running_accuracy += batch_acc

            running_error /= train_data.num_batches
            running_accuracy /= train_data.num_batches
            print('End of epoch {0:02d}: err(train)={1:.2f} acc(train)={2:.2f}'
                  .format(e + 1, running_error, running_accuracy))

            if (e + 1) % 5 == 0:
                valid_error = 0.
                valid_accuracy = 0.
                for input_batch, target_batch in valid_data:
                    batch_error, batch_acc = sess.run(
                        [error, accuracy], 
                        feed_dict={inputs: input_batch, targets: target_batch})
                    valid_error += batch_error
                    valid_accuracy += batch_acc

                valid_error /= valid_data.num_batches
                valid_accuracy /= valid_data.num_batches
                print('                 err(valid)={0:.2f} acc(valid)={1:.2f}'
                       .format(valid_error, valid_accuracy))

In [27]:
build_model(5)

{'fc-layer-4': <tf.Tensor 'fc-layer-4_2/Relu:0' shape=(?, 200) dtype=float32>, 'fc-layer-5': <tf.Tensor 'fc-layer-5/Relu:0' shape=(?, 200) dtype=float32>, 'fc-layer-6': <tf.Tensor 'fc-layer-6/Relu:0' shape=(?, 200) dtype=float32>, 'fc-layer-1': <tf.Tensor 'fc-layer-1_9/Relu:0' shape=(?, 200) dtype=float32>, 'fc-layer-2': <tf.Tensor 'fc-layer-2_3/Relu:0' shape=(?, 200) dtype=float32>, 'fc-layer-3': <tf.Tensor 'fc-layer-3_2/Relu:0' shape=(?, 200) dtype=float32>}
End of epoch 01: err(train)=1.92 acc(train)=0.30
End of epoch 02: err(train)=1.75 acc(train)=0.37
End of epoch 03: err(train)=1.67 acc(train)=0.40
End of epoch 04: err(train)=1.61 acc(train)=0.42
End of epoch 05: err(train)=1.56 acc(train)=0.44
                 err(valid)=1.59 acc(valid)=0.43


In [6]:
inputs = tf.placeholder(tf.float32, [None, train_data.inputs.shape[1]], 'inputs')
targets = tf.placeholder(tf.float32, [None, train_data.num_classes], 'targets')
num_hidden = 200

with tf.name_scope('fc-layer-1'):
    hidden_1 = fully_connected_layer(inputs, train_data.inputs.shape[1], num_hidden)
with tf.name_scope('fc-layer-2'):
    hidden_2 = fully_connected_layer(hidden_1, num_hidden, num_hidden)
with tf.name_scope('output-layer'):
    outputs = fully_connected_layer(hidden_2, num_hidden, train_data.num_classes, tf.identity)

with tf.name_scope('error'):
    error = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(outputs, targets))
with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(
            tf.equal(tf.argmax(outputs, 1), tf.argmax(targets, 1)), 
            tf.float32))

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer().minimize(error)
    
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    for e in range(100):
        running_error = 0.
        running_accuracy = 0.
        
        for input_batch, target_batch in train_data:
            _, batch_error, batch_acc = sess.run(
                [train_step, error, accuracy], 
                feed_dict={inputs: input_batch, targets: target_batch})
            
            running_error += batch_error
            running_accuracy += batch_acc
            
        running_error /= train_data.num_batches
        running_accuracy /= train_data.num_batches
        print('End of epoch {0:02d}: err(train)={1:.2f} acc(train)={2:.2f}'
              .format(e + 1, running_error, running_accuracy))
        
        if (e + 1) % 5 == 0:
            valid_error = 0.
            valid_accuracy = 0.
            for input_batch, target_batch in valid_data:
                batch_error, batch_acc = sess.run(
                    [error, accuracy], 
                    feed_dict={inputs: input_batch, targets: target_batch})
                valid_error += batch_error
                valid_accuracy += batch_acc
                
            valid_error /= valid_data.num_batches
            valid_accuracy /= valid_data.num_batches
            print('                 err(valid)={0:.2f} acc(valid)={1:.2f}'
                   .format(valid_error, valid_accuracy))